<a href="https://colab.research.google.com/github/JayJay95/JavaByteCodeGenerator/blob/master/VulnerabilityClassifier_SimpleCNNwithDataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/After4thYear/Belfast/MSc\ Cybersec/Research\ Project/Colab Notebooks

/content/drive/My Drive/After4thYear/Belfast/MSc Cybersec/Research Project/Colab Notebooks


In [0]:
import os
import re
import fnmatch
import argparse
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import test_cases
from copy import deepcopy
import pdb
import time

In [0]:
clean_opseq_folder = '/content/drive/My Drive/After4thYear/Belfast/MSc Cybersec/Research Project/Colab Notebooks/Opseq/Clean_Opseq'
vuln_opseq_folder = '/content/drive/My Drive/After4thYear/Belfast/MSc Cybersec/Research Project/Colab Notebooks/Opseq/Vuln_Opseq'

In [5]:
parser = argparse.ArgumentParser(description='Vulnerability Classifier')
# parser.add_argument('--max_opcode_seq_len', action='store', type=int, help='use different versions of network', default=8192)
# parser.add_argument('--min_opcode_seq_len', action='store', type=int, help='use different versions of network', default=32)
parser.add_argument('--lr', action='store', type=float, help='use different versions of network', default=1e-3)
parser.add_argument('--epochs', action='store', type=int, help='use different versions of network', default=10)
opt = parser.parse_args('')
print(opt)

Namespace(epochs=10, lr=0.001)


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
def find_files(files, dirs=[], extensions=[]): # recursively find files in directories
    new_dirs = []
    for d in dirs:
        try:
            new_dirs += [ os.path.join(d, f) for f in os.listdir(d)] # check in all directories except testcasesupport 
        except OSError:
            if os.path.splitext(d)[1] in extensions:
                files.append(d)

    if new_dirs:
        find_files(files, new_dirs, extensions)
    else:
        return  

def read_file(filename):
    opcode_count = 0
    line_list = []
    with open(filename, mode='rt', encoding='utf8') as f:
        content = f.readlines()        
    for line in content:
        opcode_seq = []     
        for c in range(0, len(line) - 1, 2):
            #print(line[c:(c+2)],int(line[c:(c+2)], 16))
            opcode_seq.append(int(line[c:(c+2)], 16) + 1) # add one here so that the zero'th embedding is reserved for 'blank' i.e. no instruction whatsoever not even no-op			
            opcode_count += 1
            # to save training time we only read 
            # the first opt.max_opcode_seq_len opcodes of each file
            
            # if opcode_count >= opt.max_opcode_seq_len:
            #     return opcode_seq
        line_list.append(opcode_seq)
    return line_list

def read_dataset():
    vuln = []
    clean = []
    clean_opseq_files = []
    vuln_opseq_files = []
    # min_file_len = opt.min_opcode_seq_len #ignore opcode seq files shorter than this
    find_files(clean_opseq_files, dirs=[clean_opseq_folder], extensions=['.clean'])
    for clean_file_pathname in clean_opseq_files:
        tmp = read_file(clean_file_pathname)
        # if len(tmp) >= min_file_len:
        clean.append(tmp)
    
    find_files(vuln_opseq_files, dirs=[vuln_opseq_folder], extensions=['.vuln'])
    for vuln_file_pathname in vuln_opseq_files:
        tmp = read_file(vuln_file_pathname)
        # if len(tmp) >= min_file_len:
        vuln.append(tmp)
    
    # flatten vuln and clean lists
    new_vuln = []
    for x in vuln:
        for y in x:
            new_vuln.append(y)
    
    new_clean = []
    for x in clean:
        for y in x:
            new_clean.append(y)
    return new_vuln, new_clean

In [0]:
def split_dataset(vuln, clean):
    #split the dataset into train, val, test sets
    #return the concatenated dataset and
    #indicies pointing to the train,val,test samples

    vuln_label = 0
    clean_label = 1
    dataset = deepcopy(clean) + deepcopy(vuln) # concatenate original clean and original vuln samples
    trial_dataset = clean + vuln
    for list in trial_dataset:
        if len(list) < 1000:
            list.extend([0] * (1000- len(list)))
    
    final_vuln_list = []
    for vuln_list in vuln:
        inner_vuln_list = []
        inner_vuln_list.append(vuln_list)
        inner_vuln_list.append(vuln_label)
        final_vuln_list.append(inner_vuln_list)
             
    final_clean_list = []
    for clean_list in clean:
        inner_clean_list = []
        inner_clean_list.append(clean_list)
        inner_clean_list.append(clean_label)
        final_clean_list.append(inner_clean_list)
    
    # split vuln samples randomly
    first_vuln_split = int(0.8 * len(final_vuln_list))
    second_vuln_split = int(0.1 * len(final_vuln_list))
    third_vuln_split = int(len(final_vuln_list) - (first_vuln_split + second_vuln_split))
    vuln_training_dataset, vuln_validation_dataset, vuln_testing_dataset = torch.utils.data.random_split(final_vuln_list, [first_vuln_split, second_vuln_split, third_vuln_split])

    # split clean samples randomly
    first_clean_split = int(0.8 * len(final_clean_list))
    second_clean_split = int(0.1*len(final_clean_list))
    third_clean_split = int(len(final_clean_list) - (first_clean_split + second_clean_split))
    clean_training_dataset, clean_validation_dataset, clean_testing_dataset = torch.utils.data.random_split(final_clean_list, [first_clean_split, second_clean_split, third_clean_split])

    # merge both vuln & clean training sets
    training_set = clean_training_dataset + vuln_training_dataset

    # get indices and labels from training set
    train_inds=[]
    train_labels=[]
    training_counter = 0
    
    for list in training_set:
        train_inds.append(training_counter)
        training_counter+=1
        train_labels.append(list[1])
        
    # merge both vuln & clean validation sets
    validation_set = clean_validation_dataset + vuln_validation_dataset

    # get indices and labels from validation set
    val_inds = []
    val_labels = []
    training_set_length = len(training_set)
    for list in validation_set:
        val_inds.append(training_set_length)
        training_set_length += 1
        val_labels.append(list[1])

    # merge both vuln & clean testing sets
    testing_set = clean_testing_dataset + vuln_testing_dataset

    # get indices and labels from testing set
    test_inds = []
    test_labels = []
    validation_set_length = len(training_set) + len(validation_set)
    for list in testing_set:
        test_inds.append(validation_set_length)
        validation_set_length += 1
        test_labels.append(list[1])
    # pdb.set_trace()
    return trial_dataset, training_set, validation_set, testing_set


In [0]:
class VulnerabilityDetectorNetwork(nn.Module):
    def __init__(self):
        super(VulnerabilityDetectorNetwork, self).__init__()

        self.num_of_embeddings = 205
        self.embedding_dimension = 8
        self.channels_in = 1
        self.channels_out = 64
        self.hidden_nodes = 16
        self.kernel_height_dimension = 8
        self.kernel_width_dimension = 8
        self.padding_height_dimension = 0
        self.padding_width_dimension = 0
        self.features_out = 2

        self.emb1 = nn.Embedding(self.num_of_embeddings, self.embedding_dimension)
        self.conv1 = nn.Conv2d(self.channels_in, self.channels_out, kernel_size=(self.kernel_height_dimension, self.kernel_width_dimension),
            padding=(self.padding_height_dimension, self.padding_width_dimension))
        self.lin1 = nn.Linear(self.channels_out, self.hidden_nodes)
        self.lin2 = nn.Linear(self.hidden_nodes, self.features_out)
        self.drop_out = nn.Dropout(0.5)

    def forward(self, x):
        x = self.emb1(x)
#         print(x.shape)
        x = x.unsqueeze(0)
#         print(x.shape)
        x = self.conv1(x)
#         print(x.shape)
        x = F.relu(x)
#         print(x.shape)
        x = self.drop_out(x)
#         print(x.shape)
        x = torch.max(x,2)[0]
#         print(x.shape)
        x = x.squeeze(2)
#         print(x.shape)
        x = self.lin1(x)
#         print(x.shape)
        x = self.lin2(x)
#         print(x.shape)
        
        return x

In [0]:
def evaluate_network_performance(predictions, ground_truth):
    # given an array with the predicted values and the correct values
    # calculate the precision, recall and f-score
    cm = get_confusion_matrix(predictions, ground_truth)
    TP = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    TN = cm[1][1]

    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    classification_accuracy = (TP+TN) / (TP + TN + FP + FN)
    f_score = 2 * ((precision*recall)/(precision + recall))

    return precision, recall, f_score, classification_accuracy

In [0]:
def get_confusion_matrix(preds, truth):
    K = len(np.unique(truth)) # Number of classes 
    result = np.zeros((K, K))
    for i in range(len(truth)):
        result[preds[i]][truth[i]] += 1
    confusion_matrix = result
    return confusion_matrix

In [0]:
def test_network(net, tvtLoader):
    
    criterion = nn.CrossEntropyLoss()

    test_loss = 0
    predictions = []
    labels = []

    with torch.no_grad():
#         for i in range(len(inds)):
        for i, (imgs, lbls) in enumerate(tvtLoader):
            net.eval()
            imgs = torch.stack(imgs)
            i_list = imgs.tolist()
            flat_list = [item for sublist in i_list for item in sublist]
            images_tensor = torch.LongTensor(flat_list)
            
            output = net(images_tensor.unsqueeze(0))
            val,idx = torch.max(output,1)
            predictions.append(idx.item())
            labels.append(lbls)

            test_loss = criterion(output, lbls).item()

        precision, recall, f_score, classification_accuracy = evaluate_network_performance(predictions, labels)

    return precision, recall, f_score, classification_accuracy


In [0]:
def train_network(net, trainLoader, valLoader):
    
    optimizer = optim.SGD(net.parameters(), lr=opt.lr)
    criterion = nn.CrossEntropyLoss()

    num_training_epochs = opt.epochs


    for e in range(num_training_epochs):
        randperm = torch.randperm(len(trainLoader))
        running_loss = 0

        for i, (images, labels) in enumerate(trainLoader):
            net.train()
            optimizer.zero_grad()

            images = torch.stack(images)
            i_list = images.tolist()
            flat_list = [item for sublist in i_list for item in sublist]
            images_tensor = torch.LongTensor(flat_list)
            
            net_output = net(images_tensor.unsqueeze(0))
            loss = criterion(net_output, labels)
            running_loss += loss.item()
            
            loss.backward()
            optimizer.step()

        epoch_loss = running_loss/len(trainLoader)

        print('iteration ', e, 'loss ', epoch_loss)
        print('          prec rec  f1   acc')

        precision, recall, f_score, classification_accuracy = test_network(net, valLoader)
        print('val set  ',"{0:.2f}".format(precision),"{0:.2f}".format(recall),"{0:.2f}".format(f_score),"{0:.2f}".format(classification_accuracy))

        precision, recall, f_score, classification_accuracy = test_network(net, trainLoader)
        print('train set',"{0:.2f}".format(precision),"{0:.2f}".format(recall),"{0:.2f}".format(f_score),"{0:.2f}".format(classification_accuracy))
        print()

    return net


In [0]:
def createLoaders(train_set, val_set, test_set):
    trainLoader = torch.utils.data.DataLoader(train_set, batch_size = 1, shuffle = True)
    valLoader = torch.utils.data.DataLoader(val_set, batch_size = 1, shuffle = True)
    testLoader = torch.utils.data.DataLoader(test_set, batch_size = 1, shuffle = False)    
    
    return trainLoader, valLoader, testLoader


In [0]:
def main():
    # create_clean_opseq_files() # should only run the first time this code is ran
    # print('Created clean opseq')
    # create_vuln_opseq_files() # should only run the first time this code is ran
    # print('Created vuln opseq')
    vuln, clean = read_dataset()

    dataset, train_set, val_set, test_set = split_dataset(vuln, clean)
    trainLoader, valLoader, testLoader = createLoaders(train_set, val_set, test_set)
    net = VulnerabilityDetectorNetwork()
    # net.to(device)
    print(net)
    print('started training network')
    before = time.time()
    net = train_network(net, trainLoader, valLoader)
    
    print('started testing network')
    precision, recall, f_score, classification_accuracy = test_network(net, testLoader)
    print('test set',"{0:.2f}".format(precision),"{0:.2f}".format(recall),"{0:.2f}".format(f_score),"{0:.2f}".format(classification_accuracy))
    print()
    eval_time = time.time() - before
    print ("Time to run: ", eval_time)


In [16]:
# if test_case_result_eval_function and test_case_result_split_dataset and test_case_result_network:
# if test_case_result_eval_function and test_case_result_split_dataset:
main()
# else:
#     print('one or more test cases failed - quitting')
#     quit()

VulnerabilityDetectorNetwork(
  (emb1): Embedding(205, 8)
  (conv1): Conv2d(1, 64, kernel_size=(8, 8), stride=(1, 1))
  (lin1): Linear(in_features=64, out_features=16, bias=True)
  (lin2): Linear(in_features=16, out_features=2, bias=True)
  (drop_out): Dropout(p=0.5)
)
started training network
iteration  0 loss  0.23158756157847843
          prec rec  f1   acc
val set   0.91 0.97 0.94 0.94
train set 0.91 0.97 0.94 0.94

iteration  1 loss  0.1680671176919693
          prec rec  f1   acc
val set   0.92 0.96 0.94 0.94
train set 0.92 0.96 0.94 0.94

iteration  2 loss  0.15083621893371532
          prec rec  f1   acc
val set   0.93 0.96 0.94 0.94
train set 0.93 0.96 0.94 0.94

iteration  3 loss  0.14021341492118472
          prec rec  f1   acc
val set   0.93 0.96 0.95 0.95
train set 0.94 0.96 0.95 0.95

iteration  4 loss  0.13294342565732845
          prec rec  f1   acc
val set   0.94 0.95 0.95 0.95
train set 0.94 0.95 0.95 0.95

iteration  5 loss  0.12866214948588706
          prec rec  f1